This project / notebook is created for the Toronto neighborhood assignment

In [24]:
import numpy as np 

import pandas as pd

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # comment after first run
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # comment after first run for foursquare apis
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    ------------------------------------------------------------
                       

In [10]:
# reading the wikipedia page and getting the  postal codes of canada data into a dataframe

from pandas.io.html import read_html
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = read_html(page, attrs={"class":"wikitable"})[0]
df.shape

(180, 3)

At this point, we have read the wikipedia page and got the table into the dataframe df.
the code in the next cell removes the rows which have Borough as Not assigned and the remaining rows are stored in df itself.

In [11]:
df = df[df['Borough'] != "Not assigned"]
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


The code in the below cell checks if there are any rows which have Neighbourhood as Not assigned.  Please note that there are zero rows and hence we are good to go!!!

In [12]:
x = df[df['Neighbourhood'] == "Not assigned"]
x.shape

(0, 3)

In [13]:
df.shape

(103, 3)

The above code tells us that there are 103 rows of data with the 3 columns Postal code, Borough and Neighbourhood in the dataframe df.

# 2nd part of the assignment begins 
gs is the dataframe holding the geospatial data from the csv file

In [15]:
import pandas
gs = pandas.read_csv('http://cocl.us/Geospatial_data')
gs.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [19]:
#merge method is used to merge the 2 dataframes with the common column Postal code
df_gs = pd.merge(df,gs)
df_gs

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# 3rd part of the assignment begins 

Continuous problem with the Foursquare website and hence unable to use that site / apis to get the neighborhoood info and complete this part of the assignment :(

In [36]:

import folium 
  
map_toronto = folium.Map(location = [43.689574, -79.405678], 
                                        zoom_start = 10) 

for lat, lng in  zip(df_gs ['Latitude'], df_gs['Longitude']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto